In [33]:
!pip install neptune-client
!pip install neptune-tensorflow-keras
!pip install scikit-plot


In [35]:
#@title Các bước để nhúng neptune vào code của mình
#@markdown Bước 1: Đăng ký tài khoản Neptune ở đây https://neptune.ai/product                                                 

#@markdown Bước 2: Tạo một project mới ở app.neptune.ai ở đó sẽ cung cấp cho các bạn project và API token. 

#@markdown Bước 3: Sau khi deploy neptune vào ML/DL project của mình khi train xong, mọi thông tin trong quá trình train được lưu trong project ở NeptuneAI. https://app.neptune.ai/vtca2021/test/e/TES-5/monitoring

#@markdown Việc deploy neptuneAI có mức độ phức tạp cao hơn so với tensorboard nhưng đây là MLOps, các bạn chịu khó nghiên cứu sẽ có ít nhiều thuận lợi cho các bạn sau này. Chúc các bạn thành công ^^

In [34]:
import neptune.new as neptune

run = neptune.init(
    project="vtca2021/test",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGFiYzVhNi1kOWVhLTQzNzQtOTM4Mi0wZWM1ZDQyYzA1NzIifQ==",
)  # your credentials
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=10, 
          validation_data=(x_test, y_test))

for epoch in range(10):
    run["train/loss"].log(0.9 ** epoch)

run["eval/f1_score"] = 0.66

run.stop()

https://app.neptune.ai/vtca2021/test/e/TES-6
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Epoch 1/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2194 - accuracy: 0.9357 - val_loss: 0.1068 - val_accuracy: 0.9672
Epoch 2/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0953 - accuracy: 0.9712 - val_loss: 0.0978 - val_accuracy: 0.9689
Epoch 3/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0666 - accuracy: 0.9790 - val_loss: 0.0687 - val_accuracy: 0.9768
Epoch 4/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0527 - accuracy: 0.9831 - val_loss: 0.0694 - val_accuracy: 0.9785
Epoch 5/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0453 - accuracy: 0.9854 - val_loss: 0.0766 - val_accuracy: 0.9771
Epoch 6/10
18

Waiting for the remaining 14 operations to synchronize with Neptune. Do not kill this process.


All 14 operations synced, thanks for waiting!


In [32]:
import glob
import hashlib

import matplotlib.pyplot as plt
import neptune.new as neptune
import numpy as np
import pandas as pd
import tensorflow as tf
from neptune.new.integrations.tensorflow_keras import NeptuneCallback
from scikitplot.metrics import plot_roc, plot_precision_recall

# Select project
run = neptune.init(
    project="vtca2021/test",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGFiYzVhNi1kOWVhLTQzNzQtOTM4Mi0wZWM1ZDQyYzA1NzIifQ==",
    tags=['keras', 'fashion-mnist'],
    name='keras-training')

# Prepare params
parameters = {'dense_units': 128,
              'activation': 'relu',
              'dropout': 0.23,
              'learning_rate': 0.15,
              'batch_size': 64,
              'n_epochs': 30}

run['model/params'] = parameters

# Prepare dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Log data version
run['data/version/x_train'] = hashlib.md5(x_train).hexdigest()
run['data/version/y_train'] = hashlib.md5(y_train).hexdigest()
run['data/version/x_test'] = hashlib.md5(x_test).hexdigest()
run['data/version/y_test'] = hashlib.md5(y_test).hexdigest()
run['data/class_names'] = class_names

# Log example images
for j, class_name in enumerate(class_names):
    plt.figure(figsize=(10, 10))
    label_ = np.where(y_train == j)
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(x_train[label_[0][i]], cmap=plt.cm.binary)
        plt.xlabel(class_names[j])
    run['data/train_sample'].log(neptune.types.File.as_image(plt.gcf()))
    plt.close('all')

# Prepare model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(parameters['dense_units'], activation=parameters['activation']),
    tf.keras.layers.Dropout(parameters['dropout']),
    tf.keras.layers.Dense(parameters['dense_units'], activation=parameters['activation']),
    tf.keras.layers.Dropout(parameters['dropout']),
    tf.keras.layers.Dense(10, activation='softmax')
])
optimizer = tf.keras.optimizers.SGD(learning_rate=parameters['learning_rate'])
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Log model summary
model.summary(print_fn=lambda x: run['model/summary'].log(x))

# Train model
neptune_cbk = NeptuneCallback(run=run, base_namespace='metrics')

model.fit(x_train, y_train,
          batch_size=parameters['batch_size'],
          epochs=parameters['n_epochs'],
          validation_split=0.2,
          callbacks=[neptune_cbk])

# Log model weights
model.save('trained_model')
run['model/weights/saved_model'].upload('trained_model/saved_model.pb')
for name in glob.glob('trained_model/variables/*'):
    run[name].upload(name)

# Evaluate model
eval_metrics = model.evaluate(x_test, y_test, verbose=0)
for j, metric in enumerate(eval_metrics):
    run['test/scores/{}'.format(model.metrics_names[j])] = metric

# Log predictions as table
y_pred_proba = model.predict(x_test)
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred = y_pred
df = pd.DataFrame(data={'y_test': y_test, 'y_pred': y_pred, 'y_pred_probability': y_pred_proba.max(axis=1)})
run['test/predictions'] = neptune.types.File.as_html(df)

# Log model performance visualizations
fig, ax = plt.subplots()
plot_roc(y_test, y_pred_proba, ax=ax)
run['charts/ROC'] = neptune.types.File.as_image(fig)

fig, ax = plt.subplots()
plot_precision_recall(y_test, y_pred_proba, ax=ax)
run['charts/precision-recall'] = neptune.types.File.as_image(fig)
plt.close('all')

run.wait()


https://app.neptune.ai/vtca2021/test/e/TES-5
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
4431872/4422102 [==============================] - 0s 0us/step
Epoch 1/30
750/750 [==============================] - 5s 6ms/step - loss: 0.6683 - accuracy: 0.7576 - val_loss: 0.4732 - val_accuracy: 0.8278
Epoch 2/30
750/750 [==============================] - 4s 5ms/step - loss: 0.4749 - accuracy: 0.8295 - val_loss: 0.4189 - val_accuracy: 0.8432
Epoch 3/30
750/750 [==============================] - 4s 5ms/step - loss: 0.4284 - accuracy: 0.8438 - val_loss: 0.3780 - val_accuracy: 0.8605
Epoch 4/30
750/750 [==============================] - 4s 6ms/step - loss: 0.4018 - accuracy: 0.8539 - val_loss: 0.3858 - val_accuracy: 0.8577
Epoch 5/30
750/750 [==============================] - 4s 6ms/step - loss: 0.3858 - accuracy: 0.8580 - val_l